In [ ]:
import os
gpu_num = "" # Use "" to use the CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

resolution = [480,320] # increase for higher quality of renderings

# Allows to exit cell execution in Jupyter
class ExitCell(Exception):
    def _render_traceback_(self):
        pass

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

tf.random.set_seed(1) # Set global random seed for reproducibility

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import time

# Import Sionna RT components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, RadioMaterial, LambertianPattern
from sionna.constants import PI

# For link-level simulations
from sionna.channel import cir_to_ofdm_channel, subcarrier_frequencies, OFDMChannel, ApplyOFDMChannel, CIRDataset
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver
from sionna.utils import compute_ber, ebnodb2no, PlotBER
from sionna.ofdm import KBestDetector, LinearDetector
from sionna.mimo import StreamManagement

In [ ]:
# Load integrated scene
scene = load_scene("../../scenes/outdoor_accenture/accenturePOC.xml")

In [ ]:
scene.get("elm__4").radio_material = "itu_concrete"

In [ ]:
xy_center = [-5,45]
z_centers = [5.6, 1.5]

In [ ]:
sampling_frequency = 10e6
sample_m = 62.0 / sampling_frequency

c0 = sionna.SPEED_OF_LIGHT

fc = 3.7e9
scene.frequency = fc # in Hz; implicitly updates RadioMaterials

scene.synthetic_array = False # If set to False, ray tracing will be done per antenna element (slower for large arrays)

In [ ]:
da = 0.1
M = 4
N = 4

scene._clear()

# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=N,
                             num_cols=M,
                             vertical_spacing=da,
                             horizontal_spacing=da,
                             pattern="tr38901",
                             polarization="V")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=da,
                             horizontal_spacing=da,
                             pattern="dipole",
                             polarization="V")
                             #polarization="cross")

In [ ]:
transmitters = [
    {
        "name":"main_site",
        "position":[5.31,-33.21,16],
        "orientation":tf.Variable([0.0, 0.0, 0.0], dtype=tf.float32)
    },
]

In [ ]:
for i in range(len(transmitters)):
    tx = Transmitter(name=transmitters[i]["name"],
                 position=transmitters[i]["position"],
                 orientation=transmitters[i]["orientation"])
    # rx = Receiver(name=receivers[i]["name"],
    #           position=receivers[i]["position"])
    scene.add(tx)
    print(transmitters[i]["name"])
    #scene.add(rx)

In [ ]:
import numpy as np
import tensorflow as tf

# Configure an SGD optimizer
optimizer = tf.keras.optimizers.legacy.RMSprop(0.1)

# Number of training steps
num_steps = 10

# Weighting factors for the loss terms
weight_sinr = 1
weight_coverage = 1

def train_step():
    """A single training step"""
    with tf.GradientTape() as tape:
        print(tape.watched_variables())
        
        # Compute coverage of the target area
        target_cm = scene.coverage_map(
                                       cm_center=[-32,-62]+ [z_centers[1]],
                                       cm_orientation=[0,0,0],
                                       cm_size=[80,80],
                                       cm_cell_size=[0.5,0.5])
        
        # Convert to tf.float32 to ensure dtype consistency
        target_cm_tensor = target_cm.as_tensor()
        
        # Compute coverage rate
        scaling = 1e10
        rate = tf.reduce_mean(tf.math.log(1. + target_cm_tensor * scaling))
        coverage_loss = -rate  # Note the negative sign to make it a loss
        # Combine SINR loss and coverage loss
        loss = weight_coverage * coverage_loss
    # Compute gradients and apply through the optimizer
    grads = tape.gradient(loss, tape.watched_variables())
    optimizer.apply_gradients(zip(grads, tape.watched_variables()))
    return loss

for step in range(num_steps):
    loss = train_step()
    print(f"Training step {step} - Loss: {loss.numpy():.2E} - tx orientation: {scene.transmitters['main_site'].orientation.numpy()}", end='\r')
    


In [ ]:
cm_new = scene.coverage_map(cm_center=[5.31,-33.21] + [z_centers[1]],
                            cm_orientation=[0,0,0],
                            cm_size=[200,200],
                            cm_cell_size=[0.5,0.5])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

cm_0 = 43.01 + 10*np.log10(np.where(cm_new[0]==0, 1e-32, cm_new[0]))

sns.heatmap(cm_0)

# Adjust layout
plt.tight_layout()

# Display the plots
plt.show()

In [ ]:
import numpy as np

# Define the threshold
T = -90

# Create binary matrix B
B = (cm_0 > T).astype(np.float32)

sns.heatmap(B)
plt.show()

In [ ]:
cm_new.show();

In [ ]:
scene.preview(coverage_map=cm_new)

In [ ]:
import numpy as np

def calculate_sinr_matrix(signal_matrices, noise_power):
    # Convert the noise power to linear scale
    noise_linear = 10 ** (noise_power / 10)

    # Number of matrices (4 antennas) and dimensions of the matrix
    num_antennas = len(signal_matrices)
    rows, cols = signal_matrices[0].shape

    # Initialize the SINR matrix with zeros
    sinr_matrix = np.zeros((rows, cols))

    # Optionally store the index of the reference signal (strongest signal) for each point
    reference_signal_indices = np.zeros((rows, cols), dtype=int)

    # Loop through each point in the matrices
    for i in range(rows):
        for j in range(cols):
            # Extract the signal power values at this point from all antennas
            signal_values = [signal_matrices[k][i, j] for k in range(num_antennas)]
            if i == j and i == 200: print(signal_values)

            # Convert signal values from dB to linear scale
            signal_values_linear = [10 ** (sv / 10) for sv in signal_values]

            # Find the reference signal (the strongest signal)
            reference_signal_linear = max(signal_values_linear)
            reference_signal_index = signal_values_linear.index(reference_signal_linear)

            # Store the index of the reference signal (optional)
            reference_signal_indices[i, j] = reference_signal_index

            # Calculate interference (sum of all signals except the reference signal)
            interference_linear = sum(signal_values_linear) - reference_signal_linear

            # Calculate SINR at this point
            sinr_linear = reference_signal_linear / (interference_linear+noise_linear)

            if i == j and i == 200: print(reference_signal_linear, interference_linear, noise_linear, sinr_linear)

            # Convert SINR back to dB scale and store in the SINR matrix
            sinr_matrix[i, j] = 10 * np.log10(sinr_linear)

    return sinr_matrix, reference_signal_indices

sinr_matrix, _= calculate_sinr_matrix([cm_0,cm_1,cm_2,cm_3,cm_4,cm_5,cm_6], -999999999999999999)


In [ ]:
import numpy as np

# Example matrix A
A = np.array([[3, 7, 5],
              [6, 2, 8],
              [1, 9, 4]])

# Define the threshold
T = 5

# Create binary matrix B
B = (A > T).astype(int)

print("Original Matrix A:")
print(A)

print("Binary Matrix B:")
print(B)


In [ ]:
import numpy as np

# Define the threshold
T = 13

# Create binary matrix B
B = (sinr_matrix > T).astype(np.float32)

sns.heatmap(B)
plt.show()

In [ ]:
counter = 0
summ = []
for i,row in enumerate(sinr_matrix):
    if i < 300 and i > 180:
        for j,item in enumerate(row):
            if j < 580 and j > 140:
                if item > 0.5:
                    summ.append(item)
                if item > 0.5 and item < 5: print(item,i,j)
            
            

print(min(summ),max(summ),np.mean(summ))

In [ ]:
# Print the resulting SINR matrix
print("SINR Matrix (in dB):")
sns.heatmap(sinr_matrix)
plt.show()
